"""
https://keras.io/api/data_loading/image/

## Data Augmentation

Data augmentation is a technique used to increase the size and diversity of a training dataset by applying random transformations to the original images. This helps to prevent overfitting and improve the generalization ability of the model.

In this section, we will define functions that perform common data augmentation techniques such as rotation, zooming, and horizontal flipping using Keras's `ImageDataGenerator`.
"""

## Libraries

In [2]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np


ModuleNotFoundError: No module named 'keras'

## Define data augmentation parameters

In [ ]:

set_rotation_rng        = 45 # @param {type:"slider", min:0, max:180, step:5}
set_width_shift_rng     = 0.2 # @param {type:"slider", min:0, max:1, step:0.05}
set_height_shift_rng    = 0.2 # @param {type:"slider", min:0, max:1, step:0.05}
set_shear_rng           = 0.2 # @param {type:"slider", min:0, max:1, step:0.05}
set_zoom_rng            = 0.2 # @param {type:"slider", min:0, max:1, step:0.05}
set_horizontal_flip     = True # @param {type: "bool"}
set_fill_mode           = "nearest" # @param {type: "string"}

datagen = ImageDataGenerator(
    rotation_range      = set_rotation_rng,         # Rotate images by a random degree between -20 and 20
    width_shift_range   = set_width_shift_rng,
    height_shift_range  = set_height_shift_rng,
    shear_range         = set_shear_rng,
    zoom_range          = set_zoom_rng,             # Zoom in or out by a random factor between 0.9 and 1.1
    horizontal_flip     = set_horizontal_flip,      # Randomly flip images horizontally
    fill_mode           = set_fill_mode             # Fill in missing pixels after transformations
)


## Functions

In [ ]:
def augment_image(path, datagen):
  """
  Applies data augmentation to a single image.

  Args:
    image_array: A numpy array representing the image.
    datagen: An instance of ImageDataGenerator for data augmentation with the parameters specified.
  Returns:
    A numpy array representing the augmented image.
  """


  img = load_img(path)  # this is a PIL image
  x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
  x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

  # the .flow() command below generates batches of randomly transformed images
  # and saves the results to the `preview/` directory
  i = 0
  for batch in datagen.flow(x, 
                            batch_size  = 1,
                            save_to_dir = 'preview',
                            save_prefix = 'cat', 
                            save_format = 'jpeg'):
      i += 1
      if i > 20:
          break  # otherwise the generator would loop indefinitely
  return


## Examples

In [ ]:
augmented_x_train = np.array([augment_image(img) for img in x_train])
augmented_x_val = np.array([augment_image(img) for img in x_val])